In [6]:
from __future__ import division, print_function

from IPython.display import YouTubeVideo

import numpy as np

In [2]:
YouTubeVideo('QVKj3LADCnA')

Going to be solving systems of equations using **elimination** and back-substitution. If a matrix is "good" (more on that), we'll get an efficient answer. We'll need to know when it fails as well. We won't be using determinants.

Gauss came up with elimination, but it's a "natural" idea. 

\begin{array}{rcl}
    x + 2y + z & = & 2 \\
    3x + 8y + z & = & 12 \\
    4y + z & = & 2
\end{array}

Once again, a formula for solving this system of equations is: $\pmb{Ax} = \pmb{b}$

In [7]:
A = np.array(([1, 2, 1],
              [3, 8, 1],
              [0, 4, 1 ]))

b = np.array([[2],
              [12], 
              [2]])

Treat the first equation as "OK." We're going to multiply the first equation by some multiplier and subtract it from the second equation. We want to eliminate the **x** part of equation 2. We need to multiply equation 1 by 3 and subtract it from equation 2.

The **x** position in the first equation is called the *first pivot* and that row is called the *pivot row*.

In [11]:
pivot = 0
multiplier = 3

A_new = A.copy() # For now, let's not destroy the original matrix.
A_new[1] = A_new[1] - (A_new[pivot] * multiplier)
A_new

array([[ 1,  2,  1],
       [ 0,  2, -2],
       [ 0,  4,  1]])

In [8]:
A # Check that original matrix is intact

array([[1, 2, 1],
       [3, 8, 1],
       [0, 4, 1]])

Next step would be to eliminate the **x** in the third row as well, but there is no **x** in the third equation. This means that the multiplier for this step is actually 0. This requires no work, but is actually a step.

Now we consider the *second pivot*, which is in the **y** position of equation 2. In normal matrix form, this would be position [2, 2], in NumPy it's position [1, 1].

In [14]:
pivot = 1
multiplier = 2
A_new[2] = A_new[2] - (A_new[pivot] * multiplier)
A_new

array([[ 1,  2,  1],
       [ 0,  2, -2],
       [ 0,  0,  5]])

The third pivot is in the **z** position in equation 3, position [2, 2].

This matrix now has all zeros below the *diagonal*, which makes it an *upper triangular matrix*, which we'll call **U**. The entire point of elimination was to transform **A** into **U**. 

Strang says this is one of the most fundamental operations in numerical computing.

Some caveats: pivots cannot be zero. This was a good matrix, didn't have to switch rows, etc.

In [15]:
U = A_new
U

array([[ 1,  2,  1],
       [ 0,  2, -2],
       [ 0,  0,  5]])

The *determinant* of the matrix is found by multiplying the pivots of **U**. Strang is fiercely anti-determinant.

In [23]:
print(np.prod(A_new.diagonal())) # Doing the math
print(np.linalg.det(A_new)) # Using the NumPy linear algebra function

10
10.0


What if elimination fails? What if the second pivot was a zero instead of a 1 in the very first position? We'd have to swap rows around until that first pivot was non-zero.

What if the second pivot was a zero instead of a two? We would have had that if the original A[1, 1] was a 6 instead of an 8. 

Can get out of trouble if there's a non-zero below the zero. We can exchange lower rows.

What if the final pivot, which started as a 1 (and became a 5), and became a 0 with no place to exchange? -4 would have produced this.

This would have produced a 0 in the third pivot, and the matrix would not have been invertible.

# Backsubstitution

Need to bring the RHS of the equations back into play now. Looking back at our original matrix **A**, it would look something like this:

<center>
$
\pmb{A}_{aug} = 
\left[
\begin{array}{ccc|c}
1 & 2 & 1 & 2 \\
3 & 8 & 1 & 12 \\
0 & 4 & 1 & 2
\end{array}
\right]$
</center>

This is called an *augmented matrix*. We can do the same elimination steps as above, this time incorporating the RHS.

In [29]:
A_aug = np.c_[A, b]

pivot = 0
multiplier = 3

A_aug_new = A_aug.copy() # For now, let's not destroy the original matrix.
A_aug_new[1] = A_aug_new[1] - (A_aug_new[pivot] * multiplier)

pivot = 1
multiplier = 2
A_aug_new[2] = A_aug_new[2] - (A_aug_new[pivot] * multiplier)

A_aug_new

array([[  1,   2,   1,   2],
       [  0,   2,  -2,   6],
       [  0,   0,   5, -10]])

So we now have reproduced the upper triangular matrix **U** and have a new RHS, which we'll call the vector **c**.

Instead of $\pmb{Ax} = \pmb{b}$, we have $\pmb{Ux} = \pmb{c}$.

Time for backsubstitution now. We can see that the correct value for $z$ is -2. We can then plug $z$ into equation 2, so $y = 1$, making $x = 2$. You solve the system in reverse order (moving up the matrix) because it's triangular.

# Matrices

We want to recast these operations in matrix form. We'll use *elimination matrices* to do this.

In [30]:
A

array([[1, 2, 1],
       [3, 8, 1],
       [0, 4, 1]])

If we multiply a matrix by a vector (on the right hand side), we get a vector.

A 3x3 matrix multiplied by a 3x1 vector, you get a 3x1 column vector.

A matrix * a column is a column. A row * matrix is a row. Instead of a linear combination of columns, you have linear combinations of rows. 

A 1x3 row * a 3x3 matrix produces a 1x3 row vector.

In the beginning of the course, we're dealing with rows primarily (rows are equations). 

We need to find a matrix that will leave the first row of our matrix unchanged, leaves the last row unchanged, and subtracts three times row 1 from row 2.

How to figure this out? For starters, what would we do if we wanted to do nothing to the original? We could use the *identity matrix*, which is just a square matrix with ones on the diagonal and zeros everywhere else.

In [37]:
# NumPy function np.eye() with the number of ones desired 
# (or the number or rows or columns, these are all equivalent)
np.eye(3) 

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [38]:
# We'll call this elimination matrix E

E    = np.array([[1, 0, 0],
                 [-3, 1, 0],
                 [0, 0, 1]])

A_new = E.dot(A) # Take the dot product of the elimination matrix and A
A_new

array([[ 1,  2,  1],
       [ 0,  2, -2],
       [ 0,  4,  1]])

In [39]:
# Need another matrix for next step of elimination
# This will multiply the second row and subtract it from row 3

E = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, -2, 1]])

A_new = E.dot(A_new)
A_new

array([[ 1,  2,  1],
       [ 0,  2, -2],
       [ 0,  0,  5]])

Now we just need a matrix that combines all of these steps into a single matrix. For NumPy purposes, Strang's labels for these matrices are confusing (he calls them $\pmb{E}_{32}$ and $\pmb{E}_{21}$ for elimination matrices for the [3, 2] and [2,1] positions. Due to NumPy's zero-indexing, this could get us in trouble. I'll rename them to match NumPy conventions. We just need to subtract one from all of the indices.

To produce **U**, all we need to do is chain together these operations.

$\pmb{U} = \pmb{E}_{21} \times (\pmb{E}_{10} \times \pmb{A})$

Importantly, matrix multiplication is [associative](http://en.wikipedia.org/wiki/Associative_property). We can change the order of the parentheses, but we can't change the order of the matrices. So we could recast this as:

$\pmb{U} = (\pmb{E}_{21} \times \pmb{E}_{10}) \times \pmb{A}$

What if some of the preconditions for all of this weren't met (e.g., zero in a pivot position, etc.) What if we needed to exchange rows?

We can use a *permutation matrix* to exchange rows.

Example: exchanging rows 1 & 2.

Want to find the following, and basically that means we need a matrix that produces 0 of the first row, 1 of the second row, and then 1 of the first row and 0 of the second row.

<center>
$
\begin{bmatrix} 
? \; ? \\
? \; ? 
\end{bmatrix}
\; \times
\begin{bmatrix} 
a \; b \\
c \; d 
\end{bmatrix}
= 
\begin{bmatrix} 
c \; d \\
b \; a 
\end{bmatrix}
$
</center>

The permutation matrix that will do this is:

In [42]:
P = np.array([[0, 1],
              [1, 0]])

P

array([[0, 1],
       [1, 0]])

If you wanted to do this on a column basis instead of a row basis, the permutation matrix needs to go on the right of the matrix instead of the left. 

<center>
$
\begin{bmatrix} 
a \; b \\
c \; d 
\end{bmatrix}
\; \times
\begin{bmatrix} 
0 \; 1 \\
1 \; 0 
\end{bmatrix}
= 
\begin{bmatrix} 
b \; a \\
d \; c 
\end{bmatrix}
$
</center>

We can't change the order of the matrices in these operations. Matrix multiplication is associative, but it is **not** generally commutative. That is to say $\pmb{AB} \neq \pmb{BA}$. (As Strang says, you have to keep them in their "Gauss-given order."

There's actually a better way to do all this. Instead of going from **A** to **U**, we can think about how to get back to **A** from **U**. We can *invert* the steps. To do this, we need to find the inverse matrix. We want to find the matrix that produces the identity -- a matrix multiplyied by its inverse will produce the identity.

So now instead of subtracting rows, we're going to add back rows. Let's take the following matrix:

In [14]:
A = np.array([[1, 0, 0],
              [-3, 1, 0],
              [0, 0, 1]])

We want to find the matrix that, when we multiply A by that matrix, we get the identity matrix. So we need to take 3 times row 1 and add it to row 2 (that will give us [0, 1, 0] on the second row).

In [49]:
A_new = np.array([[1, 0, 0], 
                  [3, 1, 0], 
                  [0, 0, 1]])

np.array_equal(A.dot(A_new), np.eye(3))

True

The notation for an inverted matrix is $\pmb{E}^{-1}$.